<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Mortgage Calculator chatbot using Generative AI with Vantage</b>
</header>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Introduction:</b></p>
<p style = 'font-size:16px;font-family:Arial'>In the Mortgage Calculator chatbot using Generative AI demo, the combination of <b>RAG, Langchain, and LLM models</b> allows users to ask queries in layman's terms, retrieve relevant information from the Vantage tables, and generate accurate and concise answers based on the retrieved data. This integration of retrieval-based and generative-based approaches provides a powerful tool for extracting knowledge from structured sources and delivering user-friendly responses.</p>

<p style = 'font-size:16px;font-family:Arial'>In this demo we will build Generative Question-Answering using LangChain, a powerful library for working with LLMs like GPT-3.5, GPT-4, Bloom, etc. and JumpStart in ClearScape notebooks, a system is built where users can ask business questions in natural English and receive answers with data drawn from the relevant databases.</p>


<center><img src="images/header.png" alt="mortgage calc"  width=800 height=800/></center>

<br>
<p style = 'font-size:16px;font-family:Arial'>Before going any farther, let's get a better understanding of RAG, LangChain, and LLM.</p>

<ul style = 'font-size:16px;font-family:Arial'><li> <b>Retrieval-Augmented Generation (RAG):</b></li></ul>
<p style = 'font-size:16px;font-family:Arial'> &emsp;  &emsp;RAG is a framework that combines the strengths of retrieval-based and generative-based approaches in question-answering systems.It utilizes both a retrieval model and a generative model to generate high-quality answers to user queries. The retrieval model is responsible for retrieving relevant information from a knowledge source, such as a database or documents. The generative model then takes the retrieved information as input and generates concise and accurate answers in natural language.</p>

<ul style = 'font-size:16px;font-family:Arial'><li> <b>Langchain:</b></li></ul>
<p style = 'font-size:16px;font-family:Arial'> &emsp;  &emsp; Langchain is a language model developed for understanding and generating human-like text. It is designed to handle queries and requests expressed in everyday language, enabling users to ask questions in layman's terms. Langchain leverages state-of-the-art deep learning techniques to comprehend the semantics and context of user queries. It can process various types of queries, ranging from simple factual questions to complex and nuanced queries.</p>

<ul style = 'font-size:16px;font-family:Arial'><li> <b>LLM Models (Large Language Models):</b></li></ul>
<p style = 'font-size:16px;font-family:Arial'> &emsp;  &emsp; LLM models refer to the large-scale language models that are trained on vast amounts of text data.
These models, such as GPT-3 (Generative Pre-trained Transformer 3),  GPT-3.5, GPT-4, HuggingFace BLOOM, LLaMA, Google's FLAN-T5, etc. are capable of generating human-like text responses. LLM models have been pre-trained on diverse sources of text data, enabling them to learn patterns, grammar, and context from a wide range of topics. They can be fine-tuned for specific tasks, such as question-answering, natural language understanding, and text generation.
LLM models have achieved impressive results in various natural language processing tasks and are widely used in AI applications for generating human-like text responses.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>Configuring the environment</li>
    <li>Connect to Vantage</li>
    <li>Data Exploration</li>
    <li>Setup LLM</li>
    <li>Launch the Chatbot</li>
    <li>Cleanup</li>
</ol>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>1. Configuring the environment</b>

In [ ]:
%%capture
# '%%capture' suppresses the display of installation steps of the following packages
!pip install -U teradataml==17.20.0.4
!pip install -r requirements.txt --quiet

In [ ]:
# !pip show teradataml

<p style = 'font-size:16px;font-family:Arial'>
    <i>The above statements will install the required libraries to run this demo. To gain access to installed libraries after running this, restart the kernel.</i></p>

<p style = 'font-size:16px;font-family:Arial'><b>To restart the kernel, press the escape key first, then type 0 0.</b></p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>1.1 Import the required libraries</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
import os
import pandas as pd

# teradata lib
from teradataml import *

# LLM
from langchain.agents import AgentType
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.prompts import MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.chat_models import ChatOpenAI
from langchain.tools import tool

# Suppress warnings
warnings.filterwarnings("ignore")
display.max_rows = 10

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>2. Connection to Vantage and OpenAI</b>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>2.1 Get the OpenAI API key</b></p>

<p style = 'font-size:16px;font-family:Arial'>In order to utilize this demo, you will need an OpenAI API key. If you do not have one, please refer to the instructions provided in this guide to obtain your OpenAI API key: </p>

[Openai_setup_api_key_guide](..//Openai_setup_api_key/Openai_setup_api_key.md)

In [ ]:
# enter your openai api key
api_key = input(prompt="\n Please Enter Openai api key: ")

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>2.2 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
execute_sql('''SET query_band='DEMO= Mortgage_Calculator_Python.ipynb;' UPDATE FOR SESSION;''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>2.3 Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>In the cell below, we are synthetically generating a few records for the Customer and Interest tables. This will allow us to test the functionality of our Chatbot with a small set of data before loading the entire dataset.For the Customer table, we are generating 5 records with randomized names, emails, and phone numbers, account details, etc. We are also assigning a unique customer ID to each record. For the Interest table, we are generating 5 records with randomized interest rates based on minimum and maximum of credit score. We are also assigning a unique interest ID to each record.By generating this sample data, we can quickly test the application's ability to display customer information and interests.</p>

In [ ]:
for t in ["Customer", "Interest"]:
    try:
        eng.execute(f"DROP TABLE {t}")
    except:
        pass

In [ ]:
query1 = '''
CREATE TABLE Customer (
    CustomerID INT,
    FirstName VARCHAR(50),
    LastName VARCHAR(50),
    DateOfBirth DATE,
    Gender CHAR(1),
    Address VARCHAR(100),
    City VARCHAR(50),
    State VARCHAR(50),
    Country VARCHAR(50),
    Email VARCHAR(100),
    PhoneNumber VARCHAR(15),
    AccountNumber VARCHAR(20),
    AccountType VARCHAR(20),
    Balance DECIMAL(10, 2),
    BranchID INT,
    AccountStatus VARCHAR(20),
    CreditScore INT,
    Income DECIMAL(10, 2)
);
'''

query2 = '''
CREATE TABLE Interest (
    ID INT,
    MinCreditScore INT,
    MaxCreditScore INT,
    InterestRate DECIMAL(5, 2)
);
'''

query3 = '''
-- Insert sample data into the Interest table
INSERT INTO Interest (ID, MinCreditScore, MaxCreditScore, InterestRate) VALUES (1,300, 579, 4.8);
INSERT INTO Interest (ID, MinCreditScore, MaxCreditScore, InterestRate) VALUES (2,580,669, 4.3);
INSERT INTO Interest (ID, MinCreditScore, MaxCreditScore, InterestRate) VALUES (3,670, 739, 3.9);
INSERT INTO Interest (ID, MinCreditScore, MaxCreditScore, InterestRate) VALUES (4,740, 799, 3.5);
INSERT INTO Interest (ID, MinCreditScore, MaxCreditScore, InterestRate) VALUES (5,800,850, 3.0);


-- Insert sample data into the Customer table
INSERT INTO Customer (CustomerID, FirstName, LastName, DateOfBirth, Gender, Address, City, State, Country, Email, PhoneNumber, AccountNumber, AccountType, Balance, BranchID, AccountStatus, CreditScore, Income)
VALUES (1001, 'John', 'Doe', '1990-04-15', 'M', '123 Main St', 'Anytown', 'CA', 'USA', 'john.doe@example.com', '123-456-7890', 'ACCT123456', 'Savings', 5000.00, 1, 'Active', 720, 60000.00);
INSERT INTO Customer (CustomerID, FirstName, LastName, DateOfBirth, Gender, Address, City, State, Country, Email, PhoneNumber, AccountNumber, AccountType, Balance, BranchID, AccountStatus, CreditScore, Income)
VALUES (1002, 'Jane', 'Smith', '1985-09-22', 'F', '456 Elm St', 'Othertown', 'NY', 'USA', 'jane.smith@example.com', '987-654-3210', 'ACCT987654', 'Checking', 3500.50, 2, 'Active', 680, 55000.00);
INSERT INTO Customer (CustomerID, FirstName, LastName, DateOfBirth, Gender, Address, City, State, Country, Email, PhoneNumber, AccountNumber, AccountType, Balance, BranchID, AccountStatus, CreditScore, Income)
VALUES (1003, 'Mike', 'Johnson', '1992-12-05', 'M', '789 Oak St', 'Somewhere', 'TX', 'USA', 'mike.johnson@example.com', '555-123-7890', 'ACCT555123', 'Savings', 6200.75, 1, 'Active', 750, 75000.00);
INSERT INTO Customer (CustomerID, FirstName, LastName, DateOfBirth, Gender, Address, City, State, Country, Email, PhoneNumber, AccountNumber, AccountType, Balance, BranchID, AccountStatus, CreditScore, Income)
VALUES (1004, 'Sarah', 'Wilson', '1980-07-10', 'F', '101 Pine St', 'Anothercity', 'FL', 'USA', 'sarah.wilson@example.com', '321-555-9876', 'ACCT321555', 'Checking', 9000.25, 3, 'Active', 800, 80000.00);
INSERT INTO Customer (CustomerID, FirstName, LastName, DateOfBirth, Gender, Address, City, State, Country, Email, PhoneNumber, AccountNumber, AccountType, Balance, BranchID, AccountStatus, CreditScore, Income)
VALUES (1005, 'David', 'Lee', '1978-03-30', 'M', '222 Birch St', 'Yetanothercity', 'GA', 'USA', 'david.lee@example.com', '111-222-3333', 'ACCT111222', 'Savings', 4200.30, 2, 'Active', 700, 70000.00);
'''

execute_sql(query1)
execute_sql(query2)
execute_sql(query3)

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>3. Data Exploration</b>

<p style = 'font-size:16px;font-family:Arial'>The goal of the Mortgage Chatbot is to provide a user-friendly interface for individuals to quickly and easily find personalized mortgage information. The chatbot aims to streamline the mortgage process by allowing users to ask questions in natural language and receive accurate, concise answers and calculations. By integrating with Vantage, the chatbot can retrieve relevant data and perform calculations to provide users with tailored mortgage options, empowering them to make informed decisions and save time in their home buying journey. </p>

<p style = 'font-size:16px;font-family:Arial'>In our database, we have two tables: the <b>customer</b> table and the <b>interest</b> table. The customer table contains personal and financial information, while the interest table has details on credit score ranges and their corresponding interest rates.</p>

<p style = 'font-size:16px;font-family:Arial'>The data is synthetically generated, with a few rows of customer data, including name, address, email, account number, balance, income, and other relevant information.<p/>

<p style = 'font-size:16px;font-family:Arial'>Each row represents a snapshot of data captured at a specific point in time, and each column represents a different variable. The input dataset is as follows:</p>
<p style = 'font-size:16px;font-family:Arial'> 
<ol style = 'font-size:16px;font-family:Arial'>
    <li><b>Customer data</b> i.e. firstname, lastname, date of birth, address, city, account number, Balance, branch id, account status, income etc.</li>
    <li><b>Interest data</b>: min credit score, max credit score, interest rate.</li>
</ol>
</p>

<p style = 'font-size:16px;font-family:Arial'><b><i>*Please scroll down to the end of the notebook for detailed column descriptions of the dataset.</i></b></p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>3.1 Examine the Customer and Interest table</b></p>    
<p style = 'font-size:16px;font-family:Arial'>Let's look at the sample data in Customer table.</p>

In [ ]:
tdf = DataFrame(in_schema("demo_user", "Customer"))

print("Data information: \n", tdf.shape)
tdf.sort("CustomerID")

<p style = 'font-size:16px;font-family:Arial'>There are 5 records in all, and there are 18 variables.</p>
<p style = 'font-size:16px;font-family:Arial'>Now, let's look at the interest table.</p>

In [ ]:
tdf = DataFrame(in_schema("demo_user", "Interest"))

print("Data information: \n", tdf.shape)
tdf

<p style = 'font-size:16px;font-family:Arial'>Let's view that interest rate based on an individual's credit score.</p>

In [ ]:
q = """
    select c.FirstName, c.LastName, c.CreditScore, i.InterestRate from Customer c join Interest i on c.CreditScore between i.MinCreditScore and i.MaxCreditScore
    where CustomerID=1001
"""

pd.read_sql(q, eng)

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>4. Setup LLM </b>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>4.1 Connect to database using SQLAlchemy and Initialize the Large Language Model</b></p>    

<p style = 'font-size:16px;font-family:Arial'>Under the hood, LangChain uses SQLAlchemy to connect to SQL database. The SQLDatabaseToolkit can therefore be used with any SQL dialect supported by SQLAlchemy, such as Teradata Vantage, MS SQL, MySQL, MariaDB, PostgreSQL, Oracle SQL, and SQLite. Please refer to the <a href="https://docs.sqlalchemy.org/en/20/"> SQLAlchemy documentation</a> for more information about requirements for connecting to your database. The SQLDatabaseToolkit builds off of SQLDatabaseChain and is designed to answer more general questions about a database, as well as recover from errors.</p>

<p style = 'font-size:16px;font-family:Arial'><b>Important:</b> The code below establishes a database connection for data sources and Large Language Models. Please note that the solution will only work if the database connection for your sources is defined in the cell below. In addition to this, to use OpenAI models, we have to set OpenAI API key to environment variable.</p>

In [ ]:
# OpenAI API
os.environ["OPENAI_API_KEY"] = api_key

db = SQLDatabase(eng)
llm = OpenAI(temperature=0, model_name='gpt-3.5-turbo', verbose=False)

<p style = 'font-size:16px;font-family:Arial'>In this demo we will be using OpenAI’s <code>gpt-3.5-turbo</code> model as LLM. To view list of available models on OpenAI <a href='https://platform.openai.com/docs/models'>click here</a></p>

<p style = 'font-size:16px;font-family:Arial'>In OpenAI's language models, the <b>temperature</b> parameter controls the randomness of the generated text. It affects the diversity and creativity of the model's responses.</p>

<p style = 'font-size:16px;font-family:Arial'>A higher temperature value, such as 1.0 or above, increases the randomness and diversity of the generated output. This can lead to more varied and surprising responses, but it may also result in less coherence and occasional nonsensical outputs.</p>

<p style = 'font-size:16px;font-family:Arial'>On the other hand, a lower temperature value, such as 0.2 or below, reduces randomness and makes the model's output more focused and deterministic. The generated text is likely to be more conservative, sticking closely to patterns observed in the training data.</p>

<p style = 'font-size:16px;font-family:Arial'>Choosing an appropriate temperature value depends on the desired output. Higher temperatures can be useful for creative tasks or brainstorming, while lower temperatures are preferred when you need more control over the output, such as when generating specific responses or following a particular style.</p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>4.2 Setup SQLAgent</b></p> 

<p style = 'font-size:16px;font-family:Arial'>Before jumping into the chatbot, let us first understand what is an agent and why it might be preferred over a simple SQLChain.</p>

<p style = 'font-size:16px;font-family:Arial'>An agent is a component that has access to a suite of tools, including a Large Language Model (LLM). Its distinguishing characteristic lies in its ability to make informed decisions based on user input, utilizing the appropriate tools until it achieves a satisfactory answer. For example in the context of text-to-SQL, the LangChain SQLAgent will not give up if there is an error in executing the generated SQL. Instead, it will attempt to recover by interpreting the error in a subsequent LLM call and rectify the issue. Therefore, in theory, SQLAgent should outperform SQLChain in productivity and accuracy.</p>
    
<p style = 'font-size:16px;font-family:Arial'>We can think of agents as enabling tools for LLMs. Like how a human would use a calculator for maths or perform a Google search for information — agents allow an LLM to do the same thing.</p>

In [ ]:
# define the tool
@tool
def generate_sql(query: str) -> str:
    """Given an input question, first create a syntactically correct query to run, then look at the results of the query and return the answer."""
    agent_executor = create_sql_agent(
        llm=llm,
        toolkit=SQLDatabaseToolkit(db=db, llm=llm),
        verbose=True,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    )
    return agent_executor.run(query)

<p style = 'font-size:16px;font-family:Arial'>In the code above, we're initializing SQLAgent by passing in LLM, SQLDatabaseToolkit, database, and agent type as ZeroShotReactAgent. Agents use an LLM to determine the best course of action and execute it. An action can either be using a tool and observing its output, or responding to the user. <a href='https://python.langchain.com/docs/modules/agents/agent_types/'>Here</a> are the agents available in LangChain.</p>

<p style = 'font-size:16px;font-family:Arial'>The agent used here is a <b>zero-shot-react-description</b> agent. Zero-shot means the agent functions on the current action only — it has no memory. It uses the <a href='https://arxiv.org/pdf/2210.03629.pdf'>ReAct</a> framework to decide which tool to use, based solely on the tool’s description.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>5. Launch the Chatbot</b>

<p style = 'font-size:16px;font-family:Arial'>In this demo we are using ChatOpenAI model with Memory. This advanced technology allows us to store and recall conversations, enabling our chatbot to provide more personalized and informed responses.As a mortgage advisor, our chatbot is trained to assist with a wide range of loan-related inquiries. Whether you're looking to purchase a new home, refinance an existing loan, or simply have questions about the mortgage process, our chatbot is here to help.To begin, we'll set the prompt for our chatbot to work as a mortgage advisor. This will enable it to provide tailored advice and guidance based on your unique needs and circumstances.</p>

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial'><i><b>Note:</b>Please note that our chatbot is specifically designed to address questions related to mortgage loans. If you have a question outside of this scope, we kindly ask that you refrain from asking it. This will help us provide you with the most accurate and efficient assistance possible.</i></p>
</div>


In [ ]:
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory, ConversationSummaryBufferMemory

# Current user
CustomerID = 1001

tools = [generate_sql]
functions = [format_tool_to_openai_function(f) for f in tools]
model = ChatOpenAI(temperature=0, streaming=True).bind(functions=functions)
# memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
# memory = ConversationSummaryMemory(return_messages=True, memory_key="chat_history", llm=llm)
memory = ConversationSummaryBufferMemory(return_messages=True, memory_key="chat_history", llm=llm)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You will play the role of a mortgage adviser working with a client to create a mortgage quote. I will play the role of the client.""",
        ),
        (
            "system",
            """As a mortgage advisor for the bank you have direct access to the banks data for me
            generate the SQL to get the details  like Balance, Income, CreditScore, etc. if user don't know the answer or Do not enter the answer.
            automatically call the 'generate_sql' function to retrieve the answer from the database""",
        ),
        (
            "system",
            """This information should compliment any answers you gain from the me. 
            Ask a series of questions, no more than eight, sequentially - pausing between each question to wait for a response before proceeding to the next. """,
        ),
        (
            "system",
            """And then follow this format for each subsequent question. Again, wait for my response before moving on to the next. """,
        ),
        (
            "system",
            """Start the chat with greeting and ask the questions to user as mentioned below. Greet user by starting the message: 
            'Welcome to Mortgage Chatbot! I'm here to help you navigate the world of loans and mortgages. Whether you're looking to purchase a new home, refinance an existing loan, or simply have questions about the mortgage process, 
            I'm here to provide you with expert advice and answers. Let's get started!' """,
        ),
        (
            "system",
            """For example:
            First question: What is your total annual income?
            Second question: Do you have any additional income?""",
        ),
         (
            "system",
            """Total gross income of user is, annual income plus additional sources of income if user have' """,
        ),
         (
            "system",
            """Do not ask same question more then 1 time, Give user to Rate of Interest based on his/her credit score by fetching it from bank database""",
        ),
        (
            "system",
            """Examples of question and expected SQLQuery
            Question: What will be the InterestRate for customer whose CustomerID=1
            SQLQuery: SELECT i.InterestRate from Customer c join Interest i on c.CreditScore between i.MinCreditScore and i.MaxCreditScore where CustomerID=1""",
        ),
        (
            "system",
            """Calculate the maximum house price and monthly repayment for principal and interest based on a 5% APR. Assume income tax is 25% and 
            that the bank will lend no more than an 80% LTV.""",
        ),
        (
            "system",
            """"Provide a succinct summary paragraph of these details as a response to the client but don't show the method of calculation. Follow with a formatted table of monthly amounts 
            (gross income, tax obligation, mortgage obligation, expenses, remaining disposable income)""",
        ),
        (
            "system",
            f"""If the user fails to provide a response or says 'I don't know' for a question, automatically call the 'generate_sql' function to retrieve the answer from the database. The function should return the relevant answer for the question asked.
            current user: {CustomerID}""",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

chain = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_openai_functions(x["intermediate_steps"])
    )
    | prompt
    | model
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=chain, tools=tools, verbose=False, memory=memory)

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial'><i><b>Note:</b>Chatbot is accessing multiple components, including databases and LLMs. This may cause a brief delay in responses. Your patience is appreciated.</i></p>
</div>

In [ ]:
import panel as pn

pn.extension(design="material")

# clear the memory
memory.clear()

def callback(contents, user, instance):
    return agent_executor.invoke({"input": contents})["output"]


pn.chat.ChatInterface(callback=callback).servable()

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>5. Cleanup</b>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>Cleanup work tables to prevent errors next time.</p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
for t in ["Customer", "Interest"]:
    try:
        eng.execute(f"DROP TABLE {t}")
    except:
        pass

In [ ]:
remove_context()

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Dataset:</b>

**Customer Table**
- `CustomerID`: Unique row customer id
- `FirstName`: customer first name
- `LastName`: customer last name
- `DateOfBirth`: customer birth date
- `Gender` : customer gender (categorical: "M","F",)
- `Address` : customer full address
- `City`: city of customer (categorical: 'Almond','Geneva')
- `State`: state of customer (categorical: 'NY','TX')
- `Country`: country of customer (categorical: 'USA')
- `Email`: customer email
- `PhoneNumber`: customer phone number
- `AccountNumber`: customer account number
- `AccountType`: customer account type
- `AccountStatus`: customer account status (categorical: 'Active','Inactive')
- `Balance`: account balance, in dollar (numeric)
- `BranchID`: branch id
- `CreditScore`: customer credit score
- `Income`: customer's monthly income, in dollar (numeric)

**Interest Table**
- `ID`: Unique row id
- `MinCreditScore`: min credi score
- `MaxCreditScore`: max credi score
- `InterestRate`: rate if interest

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>Langchain Python reference: <a href='https://python.langchain.com/docs/get_started/introduction.html'>here</a></li>
</ul>

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">Copyright © Teradata Corporation - 2023. All Rights Reserved.</footer>